# Sektion 2 - Single Vehicle Routing

## modified from:
## https://github.com/PlanQK/Routing/blob/master/Routing%20in%20directed%20graph.ipynb

# Routing in a directed graph
Consider a directed graph with edges $i=0, ..., N$ and edge costs $C_i$. We want to find a path from a start vertex to a target vertex. Let $X_i\in\{0,1\}$ be binary variables indicating if edge $i$ is chosen along the path.

The cost function is based on the following requirements:
- The start vertex has exactly one outgoing edge.
- The start vertex has no incoming edge.
- The target vertex has exactly one incoming edge.
- The target vertex has no outgoing edge.
- Every other vertex has
    - either one or zero incoming edges.
    - either one or zero outgoing edges.
    - an equal number of incoming and outgoing edges.


## Problem definition

In [ ]:
## Cost function specification in sympy

import sympy as sym
import pandas as pd

# number of edges (Qubits)
N = sym.symbols('N')

# possible edges X_0, ..., X_{N-1} with values in {0,1}
X = sym.IndexedBase('X')

# costs for edges
C = sym.IndexedBase('C')

# penalty
P = sym.symbols('P')

# indices
i, j, k, v = sym.symbols('i j k v')

# start vertex: incoming and outgoing edges
VsI = sym.IndexedBase('VsI')    # incoming edges
LenVsI = sym.symbols('LenVsI')  # number of incoming edges
VsO = sym.IndexedBase('VsO')    # outgoing edges
LenVsO = sym.symbols('LenVsO')  # number of incoming edges

# target vertex: incoming and outgoing edges
VtI = sym.IndexedBase('VtI')    # incoming edges
LenVtI = sym.symbols('LenVtI')  # number of incoming edges
VtO = sym.IndexedBase('Vt O')   # outgoing edges
LenVtO = sym.symbols('LenVtO')  # number of incoming edges

# middle vertices: incoming and outgoing edges
# two indices: VI_(i,k) is the kth incoming edge at vertex i
# Similarly, LenVI_i is the number of incoming edges at vertex i
# the number of lists in LenVI and LenVO is the number of central vertices and must equal
LenV = sym.IndexedBase('LenV')   # number central vertices

VI = sym.IndexedBase('VI')        # incoming edges
LenVI = sym.IndexedBase('LenVI')  # number of incoming edges
VO = sym.IndexedBase('VO')        # outgoing edges
LenVO = sym.IndexedBase('LenVO')  # number of outgoing edges

cost_function = (
    sym.Sum( C[i] * X[i] , (i,0,N-1) ) +                    # costs for edge i
    P * (sym.Sum( X[VsO[i]], (i, 0, LenVsO-1) ) - 1 )**2 +  # exactly one outgoing edge at start vertex
    P * sym.Sum( X[VsI[i]], (i, 0, LenVsI-1) ) +            # no incoming edge at start vertex
    P * (sym.Sum( X[VtI[i]], (i, 0, LenVtI-1) ) - 1 )**2 +  # exactly one incoming edge at target vertex
    P * sym.Sum( X[VtO[i]], (i, 0, LenVtO-1) ) +            # no incoming edge at target vertex
    P * sym.Sum(                                            # one or zero incoming edges at each central point
        sym.Sum(  X[VI[v,i]] , (i, 0, LenVI[v]-1) ) *
        (sym.Sum(  X[VI[v,i]] , (i, 0, LenVI[v]-1) ) -1) ,
        (v, 0, LenV-1) ) +
    P * sym.Sum(                                            # one or zero outgoing edges at each central point
        sym.Sum(  X[VO[v,i]] , (i, 0, LenVO[v]-1) ) *
        (sym.Sum(  X[VO[v,i]] , (i, 0, LenVO[v]-1) ) -1) ,
        (v, 0, LenV-1) ) +
    P * sym.Sum(                                            # equal number of incoming and outgoing edges at each central point
        (sym.Sum(  X[VI[v,i]] , (i, 0, LenVI[v]-1) ) -
        sym.Sum(  X[VO[v,i]] , (i, 0, LenVO[v]-1) ))**2 ,
        (v, 0, LenV-1) ) )
cost_function

In [ ]:
## graph structure
## external input

TestN = 5                   # number of edges
TestVsI = []                # indices of incoming edges at start vertex
TestVsO = [0,1]             # indices of outgoing edges at start vertex
TestVtI = [3,4]             # indices of incoming edges at target vertex
TestVtO = []                # indices of outgoing edges at target vertex
TestVI = [[0],[1,2]]        # indices of incoming edges at central vertices
TestVO = [[2,3],[4]]        # indices of outgoing edges at central vertices

## edge costs
## external input

TestC = [2,4,1,7,3]
TestP = 10

## Data input

In [ ]:
# diese Zelle einfach über die andere setzen, wenn mit dem kleineren graph getestet werden soll.
# die untere zelle ist die, die verwendet wird.

# dieser Graph ist als Bild unter graph-G1.png zu sehen

TestN = 17                  # number of edges
TestVsI = []                # indices of incoming edges at start vertex
TestVsO = [0,1,2,3]         # indices of outgoing edges at start vertex
TestVtI = [7,11,14,15]      # indices of incoming edges at target vertex
TestVtO = [16]                # indices of outgoing edges at target vertex
TestVI = [[0,12],[1,8],[2],[5,9],[3],[4,6],[10],[13,16]]        # indices of incoming edges at central vertices
TestVO = [[4],[5,6,7],[8,9],[10,11],[12],[13,14],[15],[]]        # indices of outgoing edges at central vertices

## edge costs
#        0 1 2 3  4  5 6 7  8 9 10 11 12 13 14 15 16
TestC = [6,4,1,2, 1, 3,2,5, 2,4, 4,2, 5, 3,3, 3, 4]
TestP = 50

## Data processing

In [ ]:
## translation of graph structure into dictionaries for sympy

single_valued_dict = {
    N: TestN,
    P: TestP,
    LenVsI: len(TestVsI),
    LenVsO: len(TestVsO),
    LenVtI: len(TestVtI),
    LenVtO: len(TestVtO),
    LenV:  len(TestVI)
}

# multi-valued dictionaries
dict_VsI = { VsI[k]: TestVsI[k] for k in range(len(TestVsI)) }

dict_VsO = { VsO[k]: TestVsO[k] for k in range(len(TestVsO)) }

dict_VtI = { VtI[k]: TestVtI[k] for k in range(len(TestVtI)) }

dict_VtO = { VtO[k]: TestVtO[k] for k in range(len(TestVtO)) }

dict_LenVI = { LenVI[k]: len(TestVI[k]) for k in range(len(TestVI)) }

dict_LenVO = { LenVO[k]: len(TestVO[k]) for k in range(len(TestVO)) }

dict_VI = { VI[k, i]: TestVI[k][i] for k in range(len(TestVI)) for i in range(len(TestVI[k])) }

dict_VO = { VO[k, i]: TestVO[k][i] for k in range(len(TestVO)) for i in range(len(TestVO[k])) }

num_qubits = TestN

dict_costs = { C[k]: TestC[k] for k in range(len(TestC)) }


# definition of cost polynomial
cost_poly = sym.Poly(cost_function
                     .subs(single_valued_dict)
                     .doit()
                     .subs(dict_VsI)
                     .subs(dict_VsO)
                     .subs(dict_VtI)
                     .subs(dict_VtO)
                     .subs(dict_LenVI)
                     .subs(dict_LenVO)
                     .doit()
                     .subs(dict_VI)
                     .subs(dict_VO)
                     .subs(dict_costs)
                     ,
                     [X[i] for i in range(num_qubits)])

# um den funktionswert eines bestimmten ergebnisses zu testen, einfach für r einsetzen
# und die nächsten drei zeilen ausführen (ent-kommentieren)

#r = [1, 0, 1, 0, 1]
#X_dict = { X[i]: r[i] for i in range(len(r))}
#cost_function.subs(single_valued_dict).doit().subs(dict_VsI).subs(dict_VsO).subs(dict_VtI).subs(dict_VtO).subs(dict_LenVI).subs(dict_LenVO).doit().subs(dict_VI).subs(dict_VO).subs(dict_costs).subs(X_dict)

cost_poly

## QAOA

In [ ]:
import qiskit
from qiskit.utils import QuantumInstance
from qiskit.algorithms import QAOA, VQE
from qiskit_optimization.problems import QuadraticProgram
from qiskit_optimization.algorithms import MinimumEigenOptimizer
from qiskit.test.mock import *
from qiskit.providers.aer import AerSimulator
from qiskit.algorithms.optimizers import COBYLA, L_BFGS_B, SPSA, SLSQP

# generate qiskit's cost function
qiskit_cost_function = QuadraticProgram()

# define qiskit variables
for i in range(num_qubits):
    qiskit_cost_function.binary_var('X_' + str(i))

# specify qiskit cost function
qiskit_cost_function.minimize(
    linear = [int(cost_poly.coeff_monomial(X[i]**1)) for i in range(num_qubits)],
    quadratic = {
        ('X_'+str(i), 'X_'+str(j)): cost_poly.coeff_monomial(X[i]**1 * X[j]**1)
        for i in range(num_qubits)
        for j in range(i,num_qubits)
    })

print(qiskit_cost_function.export_as_lp_string())

In [ ]:
from qiskit.algorithms.optimizers import COBYLA, L_BFGS_B, SPSA, SLSQP # JRL

In [ ]:
maxiter = 1
optimizer = SPSA(maxiter=maxiter)
reps = 1

qaoa = QAOA(reps=reps, optimizer=optimizer, quantum_instance =
             QuantumInstance(backend=qiskit.Aer.get_backend('qasm_simulator')))
optimizer_qaoa = MinimumEigenOptimizer(qaoa)

results = []

for i in range(10):
    print("iteration ", i)
    result_qaoa = optimizer_qaoa.solve(qiskit_cost_function)
    print(result_qaoa)
    results.append(result_qaoa)

## Results

In [ ]:
qubit_results = []
for result in results:
    qubit_string = str(result.x).replace(' ', '').replace('.', '')[1:-1]
    qubit_results.append(qubit_string)


In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10,10))
plt.hist(qubit_results, bins = len(qubit_results))
plt.xticks(rotation='vertical')

plt.legend(loc='upper right')
plt.show()

In [ ]:
from qiskit_optimization.algorithms import MinimumEigenOptimizer, RecursiveMinimumEigenOptimizer, CplexOptimizer

optimizer = CplexOptimizer() if CplexOptimizer.is_cplex_installed() else None

results_classic = []

for i in range(100):
    result = optimizer.solve(qiskit_cost_function)
    results_classic.append(result)

In [ ]:
# path cost: Wegkosten
# actual_opt_cost: Optimierungskosten. Schließt Wegkosten und Penalty-Terms ein. Ist immer positiv.
# path: qubit-ergebnis
result_df_classic = pd.DataFrame(columns = ['path_cost', 'actual_opt_cost', 'path'])

for r in results_classic:
    
    path_cost = 0
    for q in range(len(r.x)):
        if r.x[q] == 1:
            path_cost += TestC[q]
    path_string = str(r.x).replace(' ', '').replace('.', '')[1:-1]

    result_df_classic = result_df_classic.append({'path_cost': path_cost, 'actual_opt_cost': r.fval + 2 * TestP, 'path': path_string}, ignore_index=True)

print("Cplex-Optimizer:")
print(result_df_classic.sort_values(by=['actual_opt_cost']))

### VQE

In [ ]:
# JRL adding VQE
from qiskit.circuit.library import TwoLocal
optimizer=SPSA(maxiter=500)
backend=qiskit.Aer.get_backend('qasm_simulator')

ry = TwoLocal(num_qubits, 'ry', 'cz', reps=5, entanglement='linear')
vqe = VQE(ry, optimizer=optimizer, quantum_instance=QuantumInstance(backend=backend))
optimizer_vqe = MinimumEigenOptimizer(vqe)

results_vqe = []

#for i in range(10):
for i in range(10): # JRL
    print("i ",i) # JRL
    result_vqe = optimizer_vqe.solve(qiskit_cost_function)
    results_vqe.append(result_vqe)
    print("result_vqe ", result_vqe) # JRL


In [ ]:
# path cost: Wegkosten
# actual_opt_cost: Optimierungskosten. Schließt Wegkosten und Penalty-Terms ein. Ist immer positiv.
# path: qubit-ergebnis
result_df_vqe = pd.DataFrame(columns = ['path_cost', 'actual_opt_cost', 'path'])

for r in results_vqe:
    
    path_cost = 0
    for q in range(len(r.x)):
        if r.x[q] == 1:
            path_cost += TestC[q]
    path_string = str(r.x).replace(' ', '').replace('.', '')[1:-1]

    result_df_vqe = result_df_vqe.append({'path_cost': path_cost, 'actual_opt_cost': r.fval + 2 * TestP, 'path': path_string}, ignore_index=True)

print("VQE:")
print(result_df_vqe.sort_values(by=['actual_opt_cost']))